In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt


In [258]:
Store = 160
w = ['Monday', 'Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'] 
data = pd.read_csv('Store'+str(Store)+'\Store'+str(Store)+'_order_15-min-cluster_20180103-20190711.csv',index_col=0)
data['BusinessDate'] = pd.to_datetime(data['BusinessDate'])
#Specify Date-Time Variable Types
date_time = ['OrderStartDateTime','NormalDateTime','BumpedDateTime','CookingDateTime','DayHalfHour','DayQuarterHour','OrderTimeMin']
for i in date_time:
    data.loc[:,i] = pd.to_datetime(data.loc[:,i], format="%Y-%m-%d %H:%M:%S.%f") 
data.head()

,StoreKey,BusinessDate,TimeKey,CheckNum,GuestCount,TableOpenMinutes,ProductKey,CourseName,IXIName,MajorCodeName,...,BumpedDateTime_fill,OnSiteOrder_fill,OffSiteOrder_fill,OnSiteItem_fill,OffSiteItem_fill,OnSiteGuest,OffSsiteItem,Class,ItemPerGuest,OrderTime
94595,160,2019-07-11,94,700,4,87.0,7771,ENTREES,Lem-Herb Roasted Chicken,SPECIAL/SOUP/SIDES,...,2019-07-11 23:43:57,2.0,0.0,3.0,0.0,0.0,0.0,7,1.0,1377
94619,160,2019-07-11,94,702,2,52.0,21829,ENTREES,Mushroom Burger,SPECIAL/SOUP/SIDES,...,2019-07-11 23:34:56,1.0,0.0,1.0,0.0,0.0,0.0,7,1.0,1196
82150,160,2019-07-11,92,693,1,50.0,7512,ENTREES,Fettucini Alfredo Chkn,PASTAS,...,2019-07-11 23:09:31,1.0,0.0,3.0,0.0,60.0,0.0,7,2.0,691
94092,160,2019-07-11,92,669,3,83.0,7252,ENTREES,Caesar Salad,SALADS,...,2019-07-11 23:05:02.000,2.0,6.0,2.0,7.0,59.0,7.0,7,1.0,902
94197,160,2019-07-11,91,675,3,63.0,24984,ENTREES,Carne Asada Stk Medallion,STEAKS/CHOPS,...,2019-07-11 22:53:36,8.0,4.0,16.0,8.0,59.0,8.0,7,1.0,1176


In [243]:
labor = pd.read_csv('Store'+str(Store)+'\Store'+str(Store)+'_employee_punch_20180103-20190711.csv')
labor['BusinessDate'] = pd.to_datetime(labor['BusinessDate'])
datetime = ['ScheduleStart','ScheduleEnd','ActualStart','ActualEnd']
for i in datetime:
    labor[i] = pd.to_datetime(labor[i])
labor['LocationName'] = labor['LocationName'].str.lower()
labor.head()

,BusinessDate,StoreKey,EmployeeNum,ScheduleStart,ScheduleEnd,ActualStart,ActualEnd,ShiftName,JobName,LaborCategoryName,LocationName
0,2018-01-10,160,31030821,2018-01-10 16:55:00,2018-01-11 00:00:00,2018-01-10 17:22:00,2018-01-11 01:39:00,Dinner,Bartender (tip credit wage),Bar,barcw1
1,2018-01-10,160,31490241,2018-01-10 16:45:00,2018-01-10 23:00:00,2018-01-10 16:51:00,2018-01-11 00:51:00,Dinner,Server (tip credit wage),Server,.din 02
2,2018-01-10,160,31490330,2018-01-10 10:00:00,2018-01-10 17:15:00,2018-01-10 10:15:00,2018-01-10 18:28:00,Lunch,Bartender (tip credit wage),Bar,barcw1
3,2018-01-10,160,31490330,2018-01-10 10:00:00,2018-01-10 17:15:00,2018-01-10 11:02:00,2018-01-10 18:28:00,Lunch,Bartender (tip credit wage),Bar,barcw1
4,2018-01-10,160,149048,2018-01-10 09:30:00,2018-01-10 16:00:00,2018-01-10 07:58:00,2018-01-10 16:11:00,Lunch,Line Cook1,Cook,sautest


In [244]:
Station = pd.Series(data['StationName'].unique()).str.lower()
s_clean = [i.replace('broiler', 'broil') for i in Station]
s_clean = [i.replace('sandwich', 'sand') for i in s_clean]
s_clean = [i.replace('apps', 'app') for i in s_clean]
s_sort = [i for i in s_clean if (' / ' not in i)] + [i for i in s_clean if (' / ' in i)]

labor['AtStation'] = None
labor['Station'] = None
labor_st = []
for i in s_sort:
    name = 'Labor_'+str(i)
    labor_st.append(name)
    if ' / ' not in i: 
        labor['Station'] = np.where(labor['LocationName'].str.contains(i),i,labor['Station'])
        labor['AtStation'] = np.where(labor['LocationName'].str.contains(i),1,labor['AtStation'])
    elif ' / ' in i: #if combo station, must contain key words from both station 
        i1,i2  = i.split(' / ')
        print(i1,i2)
        labor['Station'] = np.where(((labor['LocationName'].str.contains(i1))|(labor['LocationName'].str.contains(i2)))&(pd.isnull(labor['AtStation'])),i,labor['Station'])
        labor['AtStation'] = np.where(((labor['LocationName'].str.contains(i1))|(labor['LocationName'].str.contains(i2)))&(pd.isnull(labor['AtStation'])),1,labor['AtStation'])


broil app
pasta saute
grill fry


In [245]:
station_location = {}
for i in s_sort: 
    station_location[i] = labor[labor['Station']==i]['LocationName'].unique()
station_location

{'saute': array(['sautest', 'sautepl', 'sautefl', 'sautemd'], dtype=object),
 'salad': array(['saladst', 'saladpr', 'saladfn', 'saladmd'], dtype=object),
 'fry': array(['fryck', 'frypl'], dtype=object),
 'pizza': array(['pizzast', 'pizzafn'], dtype=object),
 'sand': array([], dtype=object),
 'broil / app': array(['broilck', 'broilpl'], dtype=object),
 'pasta / saute': array(['pastast', 'pastapl'], dtype=object),
 'grill / fry': array(['grillck', 'grillpl'], dtype=object)}

In [225]:
#Uncategorized Location
list(labor[pd.isnull(labor['AtStation'])]['LocationName'].unique())

['barcw1',
 '.din 02',
 'busd05',
 '.din 14',
 'slicer',
 '.din 13',
 'strbake',
 '.din 16',
 '.din 04',
 'upfdrn1',
 'bar 01',
 'prodrl',
 '.din 10',
 'steward',
 '.din 09',
 'scraping',
 'pprepcr',
 '.din 06',
 'portion',
 'fill1',
 'busd01',
 'bar 04',
 'napkin1',
 'oven',
 'potwash',
 'train bake',
 'cash1',
 'crbcsh1',
 'dessert 1',
 'greet1',
 'racking',
 'cleansd',
 'busd04',
 'sauceck',
 'bar 02',
 '.din 15',
 'barcw2',
 'busd03',
 '.din 07',
 'fdrun4',
 'train bar',
 'fdrun3',
 'dsfloat',
 'carts',
 '.din 08',
 '.din 03',
 'dessert 2',
 '.din 11',
 '.din 05',
 '.din 17',
 'bar 03',
 '.din 12',
 'dskrun2',
 'cash2',
 'barmw1',
 'crowd1',
 'fdrun2',
 'fdrun5',
 'dskrun1',
 'kitchfl',
 'train bus',
 'strbar',
 'train cook',
 'busd02',
 'ost',
 'no_location',
 'upcrwd1',
 'pat 02',
 'dt',
 'tg-run1',
 'bsrncpt',
 'pat 01',
 'pat 03',
 'train fd',
 'plater',
 'train server',
 'fdrun1',
 'tg-cash1',
 'busb05',
 'dessert 3',
 'train dish',
 'train steward',
 'pprepfl',
 'sb 1',
 'upc

In [226]:
labor = labor.dropna(subset=['Station'])

In [227]:
labor.sort_values(by='BusinessDate')

,BusinessDate,StoreKey,EmployeeNum,ScheduleStart,ScheduleEnd,ActualStart,ActualEnd,ShiftName,JobName,LaborCategoryName,LocationName,AtStation,Station
4,2018-01-10,160,149048,2018-01-10 09:30:00,2018-01-10 16:00:00,2018-01-10 07:58:00,2018-01-10 16:11:00,Lunch,Line Cook1,Cook,sautest,1,saute
56,2018-01-10,160,410059,2018-01-10 09:00:00,2018-01-10 16:00:00,2018-01-10 07:38:00,2018-01-10 12:00:00,Lunch,Line Cook1,Cook,saladst,1,salad
53,2018-01-10,160,399309,2018-01-10 16:00:00,2018-01-11 00:00:00,2018-01-10 16:00:00,2018-01-11 01:20:00,Dinner,Line Cook1,Cook,fryck,1,fry
50,2018-01-10,160,389281,2018-01-10 16:00:00,2018-01-10 23:30:00,2018-01-10 08:00:00,2018-01-10 15:12:00,Dinner,Line Cook1,Cook,grillck,1,grill / fry
49,2018-01-10,160,389281,2018-01-10 09:00:00,2018-01-10 16:00:00,2018-01-10 08:00:00,2018-01-10 15:12:00,Lunch,Line Cook1,Cook,broilck,1,broil / app
47,2018-01-10,160,389281,2018-01-10 09:00:00,2018-01-10 16:00:00,2018-01-10 08:00:00,2018-01-10 00:40:00,Lunch,Line Cook1,Cook,broilck,1,broil / app
37,2018-01-10,160,353644,2018-01-10 08:00:00,2018-01-10 14:00:00,2018-01-10 08:00:00,2018-01-10 15:00:00,Lunch,Prep Cook,Prep,saladpr,1,salad
32,2018-01-10,160,332344,2018-01-10 16:00:00,2018-01-10 23:45:00,2018-01-10 16:02:00,2018-01-11 00:35:00,Dinner,Line Cook1,Cook,sautest,1,saute
48,2018-01-10,160,389281,2018-01-10 16:00:00,2018-01-10 23:30:00,2018-01-10 08:00:00,2018-01-10 00:40:00,Dinner,Line Cook1,Cook,grillck,1,grill / fry
28,2018-01-10,160,318626,2018-01-10 09:00:00,2018-01-10 13:00:00,2018-01-10 10:00:00,2018-01-10 12:19:00,Lunch,Line Cook1,Cook,fryck,1,fry


In [228]:
import sqlite3

In [ ]:
order = data.drop_duplicates(subset=['StoreKey','BusinessDate','CheckNum','OrderTimeMin'])[['StoreKey','BusinessDate','CheckNum','OrderStartDateTime','OrderTimeMin']]

In [247]:
labor['BusinessDate'].unique()

array(['2018-01-10T00:00:00.000000000', '2018-01-11T00:00:00.000000000',
       '2018-01-12T00:00:00.000000000', '2018-01-13T00:00:00.000000000',
       '2018-01-14T00:00:00.000000000', '2018-01-15T00:00:00.000000000',
       '2018-01-16T00:00:00.000000000', '2018-01-17T00:00:00.000000000',
       '2018-01-18T00:00:00.000000000', '2018-01-19T00:00:00.000000000',
       '2018-01-20T00:00:00.000000000', '2018-01-21T00:00:00.000000000',
       '2018-01-22T00:00:00.000000000', '2018-01-23T00:00:00.000000000',
       '2018-01-24T00:00:00.000000000', '2018-01-25T00:00:00.000000000',
       '2018-01-26T00:00:00.000000000', '2018-01-27T00:00:00.000000000',
       '2018-01-28T00:00:00.000000000', '2018-03-11T00:00:00.000000000',
       '2018-03-12T00:00:00.000000000', '2018-03-13T00:00:00.000000000',
       '2018-03-14T00:00:00.000000000', '2018-03-15T00:00:00.000000000',
       '2018-03-16T00:00:00.000000000', '2018-03-17T00:00:00.000000000',
       '2018-03-18T00:00:00.000000000', '2018-03-19

In [248]:
order['BusinessDate'].unique()

array(['2019-07-11T00:00:00.000000000', '2019-07-10T00:00:00.000000000',
       '2019-07-09T00:00:00.000000000', '2019-07-08T00:00:00.000000000',
       '2019-07-07T00:00:00.000000000', '2019-07-06T00:00:00.000000000',
       '2019-07-05T00:00:00.000000000', '2019-07-04T00:00:00.000000000',
       '2019-07-03T00:00:00.000000000', '2019-07-02T00:00:00.000000000',
       '2019-07-01T00:00:00.000000000', '2019-06-28T00:00:00.000000000',
       '2019-06-27T00:00:00.000000000', '2019-06-26T00:00:00.000000000',
       '2019-06-25T00:00:00.000000000', '2019-06-24T00:00:00.000000000',
       '2019-06-23T00:00:00.000000000', '2019-06-22T00:00:00.000000000',
       '2019-06-21T00:00:00.000000000', '2019-06-20T00:00:00.000000000',
       '2019-06-19T00:00:00.000000000', '2019-06-18T00:00:00.000000000',
       '2019-06-17T00:00:00.000000000', '2019-06-16T00:00:00.000000000',
       '2019-06-15T00:00:00.000000000', '2019-06-14T00:00:00.000000000',
       '2019-06-13T00:00:00.000000000', '2019-06-12

In [249]:
%%time
conn = sqlite3.connect(':memory:')
order.to_sql('ORDERS', conn, index=False)
labor.to_sql('LABOR', conn, index=False)

Wall time: 3.31 s


In [250]:
qry = '''
SELECT
ORDERS.BusinessDate, ORDERS.CheckNum, ORDERS.OrderTimeMin, LABOR.Station, COUNT(LABOR.EmployeeNum) 'EmpCount'
FROM ORDERS 
JOIN LABOR ON ORDERS.BusinessDate = LABOR.BusinessDate
	AND LABOR.ActualEnd > ORDERS.OrderStartDateTime
	AND LABOR.ActualStart <= ORDERS.OrderStartDateTime
GROUP BY ORDERS.BusinessDate, ORDERS.CheckNum, ORDERS.OrderTimeMin, LABOR.Station
'''

In [251]:
%%time
EmpCount = pd.read_sql_query(qry,conn)

Wall time: 10.2 s


In [252]:
EmpCount = EmpCount.groupby(['BusinessDate','CheckNum','OrderTimeMin','Station'])['EmpCount'].mean().unstack().reset_index().fillna(0)

In [253]:
EmpCount['BusinessDate'] = pd.to_datetime(EmpCount['BusinessDate'])
EmpCount['OrderTimeMin'] = pd.to_datetime(EmpCount['OrderTimeMin'])

In [259]:
data  = data.merge(right = EmpCount, how='left', on=['BusinessDate','CheckNum','OrderTimeMin'])

In [262]:
labor_st = [i for i in station_location if len(station_location[i])!=0]

In [264]:
data[labor_st] = data[labor_st].fillna(0)

In [265]:
data.to_csv('Store'+str(Store)+'\Store'+str(Store)+'_order_15-min-cluster_20180103-20190711.csv')

In [266]:
pd.isnull(data).sum()

StoreKey                 0
BusinessDate             0
TimeKey                  0
CheckNum                 0
GuestCount               0
TableOpenMinutes         0
ProductKey               0
CourseName               0
IXIName                  0
MajorCodeName            0
MinorCodeName            0
StationName              0
SentTime                 0
OrderStartDateTime       0
NormalDateTime           0
CookingDateTime        113
BumpedDateTime           0
NatHolidayDesc           0
EmployeeKey              0
TicketTime               0
RNK                      0
PROD_RNK                 0
ORDER_RNK                0
StationKey               0
DateKey                  0
TypeofServiceNum         0
ChannelKey               0
OpenHour                 0
OpenMinute               0
CloseHour                0
                      ... 
HalfHourStart            0
QuarterHour              0
HalfHour                 0
Hour                     0
DayHalfHour              0
DayQuarterHour           0
D